In [4]:
import torch
import segmentation_models_pytorch as smp

# Returns a Linkenet model which is basically just torch.nn.Module
pspnet = smp.PSPNet(encoder_name="resnet34",
                       encoder_weights="imagenet",
                       activation="sigmoid",
                       in_channels=3)

# preprocessing input
preprocess = smp.encoders.get_preprocessing_fn('resnet34', pretrained='imagenet')

In [5]:
#Cannot be converted from torch to onnx

# Onnx input 

x = torch.randn(1,3, 320, 320)
torch_out = pspnet(x)
torch.onnx.export(pspnet,
                 x,
                 "pspnet.onnx",
                 input_names=["input"],
                 output_names=["output"])

RuntimeError: Unsupported: ONNX export of operator adaptive_avg_pool2d, since output size is not factor of input size. Please open a bug to request ONNX export support for the missing operator.

In [11]:
# Can't be converted to openvino because not able
# to convert to onnx see above RuntimeError
!python3 /opt/intel/openvino_2021.2.200/deployment_tools/model_optimizer/mo.py --input_model pspnet.onnx

Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/home/aakash/box_of_ai_tools/Semantic_segmentation/Linknet/Linknet.onnx
	- Path for generated IR: 	/home/aakash/box_of_ai_tools/Semantic_segmentation/Linknet/.
	- IR output name: 	Linknet
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	Not specified, inherited from the model
	- Mean values: 	Not specified
	- Scale values: 	Not specified
	- Scale factor: 	Not specified
	- Precision of IR: 	FP32
	- Enable fusing: 	True
	- Enable grouped convolutions fusing: 	True
	- Move mean values to preprocess section: 	None
	- Reverse input channels: 	False
ONNX specific parameters:
Model Optimizer version: 	2021.2.0-1877-176bdf51370-releases/2021/2

[ SUCCESS ] Generated IR version 10 model.
[ SUCCESS ] XML file: /home/aakash/box_of_ai_tools/Semantic_segmentation/Linkne

In [15]:
from openvino.inference_engine import IECore
ie = IECore()

# These files including pspnet.bin, pspnet.mapping, pspnet.xml are
# create after converting the onnx model to openvino through the above step
openvino_pspnet = ie.read_network(model="pspnet.xml", weights="pspnet.bin")
exec_pspnet = ie.load_network(network=openvino_pspnet, device_name="CPU", num_requests=1)
openvino_out = exec_pspnet.infer(inputs={"input": x})

In [30]:
print("torch_output:", torch_out.detach().numpy())
print("openvino_output:", openvino_out["output"])

torch_output: [[[[0.45595703 0.49180502 0.44596174 ... 0.5503879  0.47190374
    0.40828925]
   [0.29769057 0.29933342 0.5670274  ... 0.5001958  0.4011927
    0.30840465]
   [0.34815598 0.2000385  0.22177713 ... 0.43217978 0.1232594
    0.19711807]
   ...
   [0.5592885  0.25409186 0.31379828 ... 0.4630827  0.5127501
    0.34556505]
   [0.24723668 0.43517667 0.09445605 ... 0.6893436  0.148828
    0.45809978]
   [0.39588138 0.3833054  0.4019481  ... 0.19418454 0.37380263
    0.5430046 ]]]]
openvino_output: [[[[0.49653545 0.49060637 0.49634525 ... 0.47663793 0.4768426
    0.4775843 ]
   [0.48311645 0.42374438 0.46645597 ... 0.46849546 0.49018076
    0.48510835]
   [0.48626304 0.49414045 0.46335074 ... 0.48539555 0.49795604
    0.48618877]
   ...
   [0.49445114 0.43643683 0.47895372 ... 0.43283692 0.4515373
    0.47585467]
   [0.47574365 0.49004942 0.4654309  ... 0.50837505 0.47382444
    0.48991   ]
   [0.4907947  0.47500303 0.4719079  ... 0.47865865 0.47249988
    0.49195075]]]]
[[[[ 0.0